In [63]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import random
import pymorphy2


stage = 3
continents = ['Америка', 'Африка', 'Европа', 'Азия' ]
#stage == 1 -> 'Готовы начать?'
#stage == 2 -> 'Какой континент?'

def handler(event, context):
    text = 'Hello! I\'ll repeat anything you say to me.'
    if 'request' in event and \
            'original_utterance' in event['request'] \
            and len(event['request']['original_utterance']) > 0:
        text = event['request']['original_utterance']
    return {
        'version': event['version'],
        'session': event['session'],
        'response': {
            'text': text,
            'end_session': 'false'
        },
    }


def lemmatization(text):
    tokens = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    for word in tokens:
        morph.parse(word)[0].normal_form
    return tokens

def filter_countries(text):
    tokens = nltk.word_tokenize(text)
    tokens = lemmatization(tokens)

    
def create_response(event, text):
    tokens = lemmatization(text)
    buttons = []
    positive = ['да', 'нравится', 'конечно', 'ага']
    negative = ['нет', 'не нравится', 'фу', 'не-а']
    if '?' in tokens:
        idx1 = random.randint(0, len(positive)-1)
        idx2 = random.randint(0, len(negative)-1)
        if stage == 1:
            if(idx1 == 1):
                idx1 += 1
            buttons.append({'title' : positive[idx1]})
        elif stage == 2:
            for c in continents:
                buttons.append({'title' : c})
        else:
            buttons.append({'title' : positive[idx1]})
            buttons.append({'title' : negative[idx2]})
    return {
        'version': event['version'],
        'session': event['session'],
        'response': {
            'text': text,
            'buttons' : buttons,
            'end_session': 'false'
        },
    }
 
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anna_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
#список стран
countries = {
    'Франция': ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы', 'сёрфинг'],
    'Испания' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'ночная жизнь'],
    'Италия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы'],
    'Великобритания': ['Европа', 'культура', 'море', 'водоёмы',  'сёрфинг'],
    'Турция' : ['Европа', 'культура', 'отдых', 'море'],
    'Греция' : ['Европа', 'культура', 'отдых', 'море', 'ночная жизнь'],
    'Нидерланды' : ['Европа', 'культура', 'море'],
    'Швейцария' : ['Европа', 'лыжи', 'горы', 'водоёмы'],
    'Австрия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи','водоёмы'],
    'Германия' : ['Европа', 'культура', 'горы', 'лыжи', 'ночная жизнь', 'водоёмы'],
    
    
    'США' : ['Америка', 'море', 'горы', 'пустыня', 'лыжи', 'отдых', 
             'культура', 'дикая природа', 'водоёмы',  'сёрфинг'],
    'Мексика' : ['Америка', 'море', 'горы','пустыня','древность', 'ночная жизнь', 'дайвинг'],
    
    
    'Китай' : ['Азия', 'море', 'горы','культура', 'ночная жизнь','пустыня',
               'дикая природа', 'водоёмы',  'сёрфинг'],
    'Таиланд' : ['Азия', 'море', 'отдых', 'ночная жизнь','водоёмы'],
    'Сингапур' : ['Азия', 'культура', 'ночная жизнь'],
    
    'Марокко' : ['Африка', 'море', 'горы',  'пустыня', 'ночная жизнь'],
    'ЮАР' : ['Африка', 'море', 'горы', 'пустыня', 'дикая природа', 'дайвинг',  'сёрфинг'],
    'Тунис' : ['Африка', 'море', 'горы', 'пустыня', 'отдых'],
    'Египет' : ['Африка', 'море', 'пустыня', 'культура', 'отдых', 'дайвинг'],
    'Зимбабве' : ['Африка', 'горы', 'дикая природа'],
    'Алжир' : ['Африка', 'море', 'горы', 'пустыня'],
    'ОАЭ' : ['Африка', 'море', 'отдых', 'культура', 'дикая природа']
}





clasterizator = {
    
    'дикая природа' : ['дикая', 'природа', 'джунгли', "лес"],
    
    'водоёмы' : ['река', 'озеро'],
    
    'горы' : ['горы', 'водопад'],
    
    'пустыня' : ['пустыня'],
    
    'море' : ['море', 'океан'],
    
    
    
    'ночная жизнь' : ['бар', 'клуб', 'дискотека', 'тусовка', 'концерт', 'ночной'],
    
    'культура' : ['музей', 'театр', 'опера', 'балет', 'архитектура', 'церковь'],
    
    
    
    'отдых' : ['пляж', 'отель', 'бассейн', 'загорать', 'всё включено', 'отдых'],
    
    
    'лыжи' : ['лыжи', 'горнолыжный', 'сноуборд'],
    
    'дайвинг' : ['дайвинг'],
    
    'сёрфинг' : ['сёрфинг'] 
}





In [65]:
def reverse_map(m):
    d = {}
    for key, value in m.items():
        for item in value:
            d[item] = key
    return d

In [66]:
def county_mining(continents, keywords):
    candidats = {}
    for continent in continents:
        for country, features in countries.items():
            if continent in features:
                candidats[country] = features
    candidats_score = {country : 0 for country in candidats}
    max_score = 0
    for key in keywords:
        for country, features in candidats.items():
            if key in features:
                candidats_score[country] += 1
                max_score = max(candidats_score[country], max_score)
    best_countries = []
    for country, score in candidats_score.items():
        if score == max_score:
            best_countries.append(country)
    idx = 0
    if len(best_countries) > 1:
        idx = random.randint(0, len(best_countries)-1)
    return best_countries[idx]

In [67]:
result = county_mining(['Европа'], ['лыжи', 'водоёмы', 'ночная жизнь'])

In [68]:
print(result)

Германия


In [69]:
stage = 1
create_response({'version' : 1, 'session' : 1}, 'море или горы?')

{'version': 1,
 'session': 1,
 'response': {'text': 'море или горы?',
  'buttons': [{'title': 'нравится'}],
  'end_session': 'false'}}